In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import datajoint as dj
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

import seaborn as sns
sns.set_style('ticks')

import os
import sys
import inspect

p = !pwd
p = os.path.dirname(os.path.dirname(p[0]))
if p not in sys.path:
    sys.path.append(p)

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [4]:
from cnn_sys_ident.mesonet.parameters import Fit, Model, Core, Readout, RegPath
from cnn_sys_ident.mesonet.data import MultiDataset
from cnn_sys_ident.mesonet import MODELS

Connecting aecker@52.202.15.166:3306


# Main model

In [5]:
num_filters = 16
data_key = dict(data_hash='cfcd208495d565ef66e7dff9f98764da')

In [7]:
model_rel = MODELS['HermiteSparse'] * MultiDataset() & data_key \
    & 'positive_feature_weights=False AND shared_biases=False' \
    & {'num_filters_2': num_filters}
print(len(Fit() * model_rel))
val_loss, test_corr = (Fit() * model_rel).fetch(
    'val_loss', 'test_corr', order_by='val_loss', limit=5)
print('Loss: {:.1f}, avg corr: {:.3f} +/- {:.4f}'.format(val_loss[0], test_corr[0], test_corr.std()))

30
Loss: 3635.8, avg corr: 0.472 +/- 0.0053


# Control: dense feature weights

In [8]:
model_rel = MODELS['HermiteDense'] * MultiDataset() & data_key \
    & 'positive_feature_weights=False AND shared_biases=False' \
    & {'num_filters_2': num_filters}
print(len(Fit() * model_rel))
val_loss, test_corr = (Fit() * model_rel).fetch(
    'val_loss', 'test_corr', order_by='val_loss', limit=5)
print('Loss: {:.1f}, avg corr: {:.3f} +/- {:.4f}'.format(val_loss[0], test_corr[0], test_corr.std()))

32
Loss: 3863.2, avg corr: 0.271 +/- 0.0107


# Control: positive feature weights

In [9]:
model_rel = MODELS['HermiteSparse'] * MultiDataset() & data_key \
    & 'positive_feature_weights=True AND shared_biases=False' \
    & {'num_filters_2': num_filters}
print(len(Fit() * model_rel))
val_loss, test_corr = (Fit() * model_rel).fetch(
    'val_loss', 'test_corr', order_by='val_loss', limit=5)
print('Loss: {:.1f}, avg corr: {:.3f} +/- {:.4f}'.format(val_loss[0], test_corr[0], test_corr.std()))

32
Loss: 3686.5, avg corr: 0.429 +/- 0.0064


# Baseline: regular CNNs

In [12]:
cnn_filter_nums = [
    [32, 32, 32],
    [64, 64, 64],
    [128, 128, 128],
    [128, 128, 256],
]
for n in cnn_filter_nums:
    key = {'num_filters_{:d}'.format(i): n[i] for i in range(len(n))}
    model_rel = MODELS['CNNSparse'] * MultiDataset() & data_key \
        & 'positive_feature_weights=False'
    print(len(Fit() * model_rel & key))
    val_loss, test_corr = (Fit() * model_rel & key).fetch(
        'val_loss', 'test_corr', order_by='val_loss', limit=5)
    print('Features: {}, Loss: {:.1f}, Avg. corr: {:.3f} +/- {:.4f}'.format(
        n, val_loss[0], test_corr[0], test_corr.std()))

32
Features: [32, 32, 32], Loss: 3745.6, Avg. corr: 0.394 +/- 0.0089
32
Features: [64, 64, 64], Loss: 3736.2, Avg. corr: 0.394 +/- 0.0056
32
Features: [128, 128, 128], Loss: 3717.1, Avg. corr: 0.418 +/- 0.0082
32
Features: [128, 128, 256], Loss: 3722.9, Avg. corr: 0.415 +/- 0.0091
